In [ ]:
import os

In [ ]:
import geopandas as gpd
gpd.options.use_pygeos=True
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import scipy.stats as stats  

In [ ]:
os.getcwd()

In [ ]:
root = os.path.abspath(os.path.join(os.getcwd(),'..','..','..'))

In [ ]:
gdf = gpd.read_file(os.path.join(root,'data','SPV_newmw.gpkg'))

In [ ]:
gdf['capacity_mw'].sum()/1000/107.4-1

In [ ]:
gdf[['capacity_mw','iso-3166-1']].groupby('iso-3166-1').sum().sort_values('capacity_mw',ascending=False).loc['JP','capacity_mw']/1000/6.0-1

In [ ]:
table_out = gdf[['capacity_mw','iso-3166-1']].groupby('iso-3166-1').sum().sort_values('capacity_mw',ascending=False).iloc[0:91]

In [ ]:
table_out = table_out.loc[table_out.index!='',:]

In [ ]:
table_out['N_installations'] = gdf[['unique_id','iso-3166-1']].groupby('iso-3166-1').size()

In [ ]:
table_out['mean_size'] = gdf[['capacity_mw','iso-3166-1']].groupby('iso-3166-1').mean()

In [ ]:
table_out['capacity_mw'] = table_out['capacity_mw'].apply(lambda el: f'{el:,.0f}')
table_out['N_installations'] = table_out['N_installations'].apply(lambda el: f'{el:,.0f}')
table_out['mean_size'] = table_out['mean_size'].apply(lambda el: f'{el:,.2f}')

In [ ]:
table_out

In [ ]:
table_out.to_csv(os.path.join(root,'data','summary_table.csv'))

In [ ]:
gdf['capacity_mw'].sum()/1000- 107.4

In [ ]:
bins = {
    0:{'min':10**2,'max':10**3,'recall':.18},
    1:{'min':10**3,'max':10**4,'recall':.25},
    2:{'min':10**4,'max':10**5,'recall':.71},
    3:{'min':10**5,'max':10**6,'recall':.82},
    4:{'min':10**6,'max':10**10,'recall':.88},
}

In [ ]:
for kk,vv in bins.items():
    gdf.loc[(gdf['area']>=vv['min'])&(gdf['area']<vv['max']),'capacity_mw'] = gdf.loc[(gdf['area']>=vv['min'])&(gdf['area']<vv['max']),'capacity_mw'] /vv['recall']

In [ ]:
gdf['capacity_mw'].sum()

In [ ]:
#eia = pd.read_csv(os.path.join(root,'data','eia_powerstations','february_generator2020.csv'))
eia = pd.read_csv(os.path.join(root,'data','tabula-NREL.csv'))

In [ ]:
eia.loc[~eia['Module efficiency'].isna(),'Module efficiency'] = eia.loc[~eia['Module efficiency'].isna(),'Module efficiency'].str.replace('%','').astype(float)/100

In [ ]:
eff_alpha, eff_loc, eff_beta = stats.gamma.fit(eia.loc[~eia['Module efficiency'].isna(),'Module efficiency'].values.tolist())

In [ ]:
ilr_alpha, ilr_loc, ilr_beta = stats.gamma.fit(eia.loc[(~eia['ILR'].isna()) & (eia['ILR']!=1) & (eia['ILR']<1.7),'ILR'].astype(float).values.tolist())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def fitgamma(xdata, alpha, loc, beta):
    data = sorted(stats.gamma.rvs(alpha, loc=loc, scale=beta, size=10000))
    mean, var, skew, kurt = stats.gamma.stats(alpha, loc=loc, scale=beta, moments='mvsk')
    return [data[2500],mean,data[5000],data[7500]]

In [ ]:
curve_fit(fitgamma,5,[.44, .47, .51, .58], p0=[2,0,0.25])

In [ ]:
stats.gamma.cdf(.75, 1, loc=0, scale=1)

In [ ]:
targets = {'25':.44,'50':.47,'mean':.51,'75':.58}
arr = np.zeros((50,50))
x_off, x_step = 0.01, 0.001
y_off, y_step = 18, 0.1
for x in [x for x in range(50)]:
    for y in [y for y in range(50)]:
        loc = x_off+x*x_step
        alpha = y_off+y*y_step
        beta =(.51-loc)/alpha
        mean, var, skew, kurt = stats.gamma.stats(alpha, loc=loc, scale=beta, moments='mvsk')
        data = sorted(stats.gamma.rvs(alpha, loc=loc, scale=beta, size=10000))
        res = {'25':data[2500],'50':data[5000],'mean':mean,'75':data[7500]}
        err = sum([(res[kk]-targets[kk])**2 for kk in targets.keys()])
        arr[x,y] = err
        #print (loc, alpha, err)

In [ ]:
arr[16,21]

In [ ]:
plt.imshow(arr)

In [ ]:
np.where(arr<0.0009)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,6))
alpha = 19.5 #20.1
loc = 0.030#26
beta =(.51-loc)/alpha
mean, var, skew, kurt = stats.gamma.stats(alpha, loc=loc, scale=beta, moments='mvsk')
data = sorted(stats.gamma.rvs(alpha, loc=loc, scale=beta, size=10000))
ax.hist(data, bins=30)
plt.show()
targets = {'25':.44,'50':.47,'mean':.51,'75':.58}
res = {'25':data[2500],'50':data[5000],'mean':mean,'75':data[7500]}
for kk in targets.keys():
    print (kk, targets[kk],res[kk])

In [ ]:
import numpy as np

In [ ]:
gdf['cap_test'] = gdf['area'] * gdf['GCR'] * gdf['gti'] * gdf['eff'] / gdf['pvout'] / gdf['ILR']

In [ ]:
compiled_final 30 {'P': nan, 'P_TP': 0, 'P_FP': 0, 'R': 0.021739130434782608, 'R_TP': 2, 'R_FN': 90}
compiled_final 100.0 {'P': 0.9666666666666667, 'P_TP': 87, 'P_FP': 3, 'R': 0.0385739333722969, 'R_TP': 66, 'R_FN': 1645}
compiled_final 1000.0 {'P': 0.959917780061665, 'P_TP': 934, 'P_FP': 39, 'R': 0.42683881064162754, 'R_TP': 1091, 'R_FN': 1465}
compiled_final 10000.0 {'P': 0.9893858984078847, 'P_TP': 1305, 'P_FP': 14, 'R': 0.8830097087378641, 'R_TP': 1819, 'R_FN': 241}
compiled_final 100000.0 {'P': 0.9811676082862524, 'P_TP': 521, 'P_FP': 10, 'R': 0.9650959860383944, 'R_TP': 553, 'R_FN': 20}
compiled_final 1000000.0 {'P': 0.9722222222222222, 'P_TP': 70, 'P_FP': 2, 'R': 0.9393939393939394, 'R_TP': 62, 'R_FN': 4}

In [ ]:
0 1.1543370313553463 0.0 2
1 0.37701015880980093 0.6370519408706289 66
2 0.1994739505469657 0.7217854444253359 1091
3 0.10893815502345362 0.5129571834277274 1819
4 -0.012558514210915256 0.28063800305152226 553
5 -0.024982255186530276 0.20184961361379802 62

In [ ]:
bins = {
    0:{'min':10**2,'max':10**3,'recall_T':66, 'recall_F':1645, 'precision_T':87,'precision_F':3, 'area_mu':0.37701015880980093, 'area_std': 0.6370519408706289},
    1:{'min':10**3,'max':10**4,'recall_T':1091, 'recall_F':1465, 'precision_T':934,'precision_F':39, 'area_mu':0.1994739505469657,'area_std': 0.7217854444253359},
    2:{'min':10**4,'max':10**5,'recall_T':1819, 'recall_F':241, 'precision_T':1305,'precision_F':14, 'area_mu':0.10893815502345362,'area_std': 0.5129571834277274},
    3:{'min':10**5,'max':10**6,'recall_T':553, 'recall_F':20, 'precision_T':521,'precision_F':10, 'area_mu':-0.012558514210915256, 'area_std': 0.28063800305152226},
    4:{'min':10**6,'max':10**10,'recall_T':62, 'recall_F':4, 'precision_T':70,'precision_F':2, 'area_mu':-0.024982255186530276, 'area_std': 0.20184961361379802},
}

In [ ]:
gcr_alpha = 19.5 #20.1
gcr_loc = 0.030#26
gcr_beta =(.51-loc)/alpha

In [ ]:
gcr_beta

In [ ]:
data = sorted(stats.gamma.rvs(gcr_alpha, loc=gcr_loc, scale=gcr_beta, size=10000))
plt.hist(np.array(data)+0.15, bins=30)

In [ ]:
print (f'gcr,{gcr_alpha:.2f}, {gcr_loc:.5f}, {gcr_beta:.7f}')
print (f'eff,{eff_alpha:.2f}, {eff_loc:.5f}, {eff_beta:.7f}')
print (f'ilr,{ilr_alpha:.2f}, {ilr_loc:.5f},{ilr_beta:.7f}')

In [ ]:
best_est = []
simple_est = []
vals = []

# sample precision and recall using binomial distibrution.
N = 1000

# stats.binom.rvs(n=200,p=0.97,size=10) 
# prepopulate recalls and precisions

pm_gcr = 0.2

gdf['area_error'] = 1 # default to 1
recalls = {}
precisions = {}
area_errors = {}
for kk,vv in bins.items():
    recalls[kk] = (stats.binom.rvs(n=(vv['recall_T']+vv['recall_F']),p=vv['recall_T']/(vv['recall_T']+vv['recall_F']),size=N)/(vv['recall_T']+vv['recall_F'])).clip(0,1)
    precisions[kk] = stats.binom.rvs(n=(vv['precision_T']+vv['precision_F']),p=vv['precision_T']/(vv['precision_T']+vv['precision_F']),size=N)/(vv['precision_T']+vv['precision_F'])
    area_errors[kk] = stats.norm.rvs(vv['area_mu'],vv['area_std'],size=N)
    
for ii in range(N):
    if ii % 100 == 0:
        print (ii)
    # add GCR area error
    for kk,vv in bins.items():
        size = gdf.loc[((gdf['area']>=vv['min'])&(gdf['area']<vv['max'])),'area_error'].size
        gdf.loc[((gdf['area']>=vv['min'])&(gdf['area']<vv['max'])),'area_error'] = 1 + stats.norm.rvs(vv['area_mu'],vv['area_std'],size=size).clip(-1,None)
        #gdf.loc[((gdf['area']>=vv['min'])&(gdf['area']<vv['max'])),'area_error'] = 1 + area_errors[kk][ii]
    
    gdf['GCR'] = stats.gamma.rvs(gcr_alpha, loc=gcr_loc, scale=gcr_beta, size=len(gdf))
    #for iso2 in gdf['iso-3166-1'].unique():
    #    size = len(gdf.loc[gdf['iso-3166-1']==iso2,:])
    #    gcr_err = np.random.rand()*2*pm_gcr - pm_gcr if iso2!='US' else 0
    #    gdf.loc[gdf['iso-3166-1']==iso2,'GCR'] = np.array(stats.gamma.rvs(gcr_alpha, loc=gcr_loc, scale=gcr_beta, size=size))+gcr_err
    
    gdf['eff'] = stats.gamma.rvs(eff_alpha, loc=eff_loc, scale=eff_beta, size=len(gdf))
    gdf['ILR'] = stats.gamma.rvs(ilr_alpha, loc=ilr_loc, scale=ilr_beta, size=len(gdf))
    gdf['cap_test'] = gdf['area'] * gdf['area_error'] * gdf['GCR'] * gdf['gti'] * gdf['eff'] / gdf['pvout'] / gdf['ILR']
    mw_test = gdf['cap_test'].sum()/1000
    el = 0
    for kk, vv in bins.items():
        #print (vv,((gdf['area']>=vv['min'])&(gdf['area']<vv['max'])).sum())
        el += gdf.loc[((gdf['area']>=vv['min'])&(gdf['area']<vv['max'])),'cap_test'].sum()*precisions[kk][ii]/recalls[kk][ii]
        
    vals.append(gdf['cap_test'].values.copy())
    best_est.append(el)
    simple_est.append(mw_test)

In [ ]:
gdf['area_error'].hist(bins=50)

In [ ]:
gdf['area_error'].min(), gdf['area_error'].max()

In [ ]:
vals = np.array(vals)

In [ ]:
vals.shape

In [ ]:
vals[0,:]

In [ ]:
vals[900,:]

In [ ]:
vals = np.sort(vals,axis=0)

In [ ]:
vals[25,:].sum(), vals[975,:].sum(), vals[500,:].sum()

In [ ]:
import pickle

In [ ]:
pickle.dump(best_est, open('./best_est.pickle','wb'))

In [ ]:
pd.read_csv(os.path.join(root,'data','tabula-irena.csv')).set_index('iso2')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
np.quantile(np.array(sorted(best_est))/1000/1000,[0.025,0.975]), (np.array(sorted(best_est))/1000/1000).mean()

In [ ]:
np.quantile(np.array(sorted(best_est))/1000/1000,[0.025,0.975]) - (np.array(sorted(best_est))/1000/1000).mean()

In [ ]:
np.quantile(np.array(sorted(simple_est))/1000,[0.025,0.975]), (np.array(sorted(simple_est))/1000).mean()

In [ ]:
plt.hist(best_est, bins=20)

In [ ]:
GCR_mean, var, skew, kurt = stats.gamma.stats(gcr_alpha, loc=gcr_loc, scale=gcr_beta, moments='mvsk')
eff_mean, var, skew, kurt = stats.gamma.stats(eff_alpha, loc=eff_loc, scale=eff_beta, moments='mvsk')
ilr_mean, var, skew, kurt = stats.gamma.stats(ilr_alpha, loc=ilr_loc, scale=ilr_beta, moments='mvsk')

In [ ]:
gdf['GCR'] = GCR_mean
gdf['eff'] = eff_mean
gdf['ILR'] = ilr_mean
for kk,vv in bins.items():
        gdf.loc[((gdf['area']>=vv['min'])&(gdf['area']<vv['max'])),'area_error'] = 1 + vv['area_mu']
gdf['capacity_mw'] = gdf['area'] * gdf['area_error'] * gdf['GCR'] * gdf['gti'] * gdf['eff'] / gdf['pvout'] / gdf['ILR'] / 1000

In [ ]:
gdf.to_file(os.path.join(root,'data','SPV_newmw.gpkg'),driver='GPKG')

In [ ]:
gdf['capacity_mw'].sum()/1000

In [ ]:
gdf

In [ ]:
eia[eia['Technology']=='Solar Photovoltaic'].to_csv(os.path.join(root,'data','eia_powerstations','pv_only.csv'))

In [ ]:
np.random.choice(10,5, replace=False)

In [ ]:
gdf.columns

In [ ]:
stats.binom.rvs(n=200,p=0.97,size=10) # n successes

In [ ]:
stats.norm.rvs(5,0.2,size=10)

### dep

In [ ]:
bins = {
    0:{'min':10**2,'max':10**3,'recall_n':.18, 'recall_p':, 'precision_n':,'precision_p':, 'area_mu': ,'area_std':},
    1:{'min':10**3,'max':10**4,'recall':.25},
    2:{'min':10**4,'max':10**5,'recall':.71},
    3:{'min':10**5,'max':10**6,'recall':.82},
    4:{'min':10**6,'max':10**10,'recall':.88},
}